In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import shuffle, class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from scipy.fft import fft, fftfreq


from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf


# Mount Drive
drive.mount('/content/drive')


# Access grip names and file paths
grip_files = {
   'Tip': '/content/drive/MyDrive/Data/Tip.csv',
   'Spherical': '/content/drive/MyDrive/Data/Spherical.csv',
   'Palmer': '/content/drive/MyDrive/Data/Palmer.csv',
   'Cylindrical': '/content/drive/MyDrive/Data/Cylindrical.csv',
   'Hook': '/content/drive/MyDrive/Data/Hook.csv',
   'Lateral': '/content/drive/MyDrive/Data/Lateral.csv',
}


# Feature extraction
def extract_features(signal, fs=500):
   features = {}
   features['MAV'] = np.mean(np.abs(signal))
   features['RMS'] = np.sqrt(np.mean(signal**2))
   features['VAR'] = np.var(signal)
   features['IEMG'] = np.sum(np.abs(signal))
   features['ZC'] = ((signal[:-1] * signal[1:]) < 0).sum()
   features['WL'] = np.sum(np.abs(np.diff(signal)))
   diff1 = np.diff(signal)
   diff2 = np.diff(diff1)
   features['SSC'] = np.sum((diff1[:-1] * diff1[1:] < 0) & (np.abs(diff2) > 0.01))
   features['WAMP'] = np.sum(np.abs(np.diff(signal)) > 0.02)
   features['LOG_D'] = np.exp(np.mean(np.log(np.abs(signal) + 1e-6)))


   # Frequency-domain features
   N = len(signal)
   freqs = fftfreq(N, 1 / fs)
   fft_vals = np.abs(fft(signal))
   positive_freqs = freqs[:N // 2]
   positive_fft = fft_vals[:N // 2]
   total_power = np.sum(positive_fft)
   features['Total_Power'] = total_power
   features['MNF'] = np.sum(positive_freqs * positive_fft) / total_power
   features['MDF'] = positive_freqs[np.where(np.cumsum(positive_fft) >= total_power / 2)[0][0]]


   return features


# Sliding window parameters
window_size = 50
step_size = 10


# Build feature dataset
feature_rows = []


for grip_label, file_path in grip_files.items():
   df = pd.read_csv(file_path, header=None)
   ch1 = df.iloc[:, 0].values
   ch2 = df.iloc[:, 1].values


   for i in range(0, len(ch1) - window_size + 1, step_size):
       window1 = ch1[i:i + window_size]
       window2 = ch2[i:i + window_size]


       feat1 = extract_features(window1)
       feat2 = extract_features(window2)


       combined = {f'EMG1_{k}': v for k, v in feat1.items()}
       combined.update({f'EMG2_{k}': v for k, v in feat2.items()})
       combined['Grip'] = grip_label
       feature_rows.append(combined)


# Convert to dataframe
features_df = pd.DataFrame(feature_rows)


# grip labels
label_encoder = LabelEncoder()
y_labels = label_encoder.fit_transform(features_df['Grip'])
y = to_categorical(y_labels, num_classes=6)


# Prepare features
X = features_df.drop(columns='Grip').values
X, y = shuffle(X, y, random_state=42)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


# Class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_labels), y=y_labels)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}


# Build model
model = Sequential([
   Dense(200, activation='relu', input_dim=X_train.shape[1]),
   BatchNormalization(),
   Dropout(0.2),


   Dense(80, activation='relu'),
   BatchNormalization(),
   Dropout(0.2),


   Dense(64, activation='relu'),
   BatchNormalization(),
   Dropout(0.2),


   Dense(6, activation='softmax')
])


model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-4)


# Train
history = model.fit(
   X_train, y_train,
   validation_data=(X_test, y_test),
   epochs=15,
   batch_size=50,
   class_weight=class_weights_dict,
   callbacks=[early_stopping, reduce_lr]
)


# Predict labels
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)




# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {acc:.4f}")




# Convert to TFLite full integer quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8


def representative_dataset():
   for i in range(min(100, len(X_train))):
       yield [X_train[i:i+1].astype(np.float32)]


converter.representative_dataset = representative_dataset
tflite_quant_model = converter.convert()


# Save the quantized model
with open("grip_model_quant.tflite", "wb") as f:
   f.write(tflite_quant_model)




from google.colab import files
files.download("grip_model_quant.tflite")


# Load and evaluate quantized model
interpreter = tf.lite.Interpreter(model_path="grip_model_quant.tflite")
interpreter.allocate_tensors()


input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


# Get quantization parameters
input_scale = input_details[0]['quantization'][0]
input_zero_point = input_details[0]['quantization'][1]


# Evaluate on test data
correct = 0
total = len(X_test)


print("\nQuantization parameters:")
print(f"Input scale: {input_scale}")
print(f"Input zero point: {input_zero_point}")


for i in range(total):
   # Quantize the input data
   input_data = X_test[i].astype(np.float32)
   input_data_quantized = np.array(input_data / input_scale + input_zero_point, dtype=np.int8)
   input_data_quantized = np.expand_dims(input_data_quantized, axis=0)


   interpreter.set_tensor(input_details[0]['index'], input_data_quantized)
   interpreter.invoke()
   output_data = interpreter.get_tensor(output_details[0]['index'])


   # Dequantize the output
   output_scale = output_details[0]['quantization'][0]
   output_zero_point = output_details[0]['quantization'][1]
   output_data_dequantized = (output_data.astype(np.float32) - output_zero_point) * output_scale


   if np.argmax(output_data_dequantized) == np.argmax(y_test[i]):
       correct += 1


quant_acc = correct / total
print(f"⚡ Quantized Model Accuracy: {quant_acc:.4f}")


# Print quantization parameters for C++ code
print("\nFor C++ code:")
print(f"Input scale: {input_scale}")
print(f"Input zero point: {input_zero_point}")
print(f"Output scale: {output_scale}")
print(f"Output zero point: {output_zero_point}")




# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
grip_names = label_encoder.classes_


# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=grip_names, yticklabels=grip_names)
plt.xlabel('Predicted Grip')
plt.ylabel('True Grip')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()
